In [1]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from utils import file_to_coupling_map, directory_to_circuits

import random
import time
import os
import numpy as np


In [2]:
seed = 42
random.seed(seed)

cmap_names = ["toyko_20.txt"]
qc_name = "paper"
heuristic_list = ["lookahead"]
qc_list = directory_to_circuits(f"circuits/{qc_name}")
seed_list = [42] * len(qc_list)
pass_trials = 20

# CSV file to store the results
csv_path = f"results/{qc_name}.csv"
# Columns are: "qc_label, swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("qc_label,swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")




In [3]:
# Run on all heuristics
for heuristic in heuristic_list:
    print(f"Running {heuristic} heuristic")
    # Run on all coupling maps
    for cmap_name in cmap_names: 
        print(f"    Loading coupling map {cmap_name}")
        cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
        # Run on all circuits
        for qc in qc_list:
            qc_label = qc.name
            num_qubits = qc.num_qubits
            curr_seed = seed_list[qc_list.index(qc)]
            print(f"        Running {qc.name} on {cmap_name} with seed {curr_seed}")
            # Create the routing pass
            rp = SabreSwap(coupling_map=cmap, heuristic=heuristic, seed=curr_seed, trials=pass_trials)
            lp = SabreLayout(coupling_map=cmap, routing_pass=rp, seed=curr_seed)
            # Create the pass manager
            pm = PassManager([lp, FullAncillaAllocation(cmap), EnlargeWithAncilla(), ApplyLayout(), rp])
            # Execute the pass manager
            start = time.time()
            qc_tr = pm.run(qc)
            end = time.time()
            # Collect the metrics for this trial
            runtime = end - start
            swap_count = qc_tr.count_ops().get("swap", 0)
            depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

            # Write the results to the CSV file
            # Columns are: "qc_label, swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
            with open(csv_path, "a") as f:
                f.write(f"{qc_label},{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
            print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

    

Running lookahead heuristic
    Loading coupling map toyko_20.txt
        Running qft_10.qasm on toyko_20.txt with seed 42
            Swap count: 20, Depth: 44, Runtime: 0.04926109313964844
        Running qft_13.qasm on toyko_20.txt with seed 42
            Swap count: 41, Depth: 71, Runtime: 0.07844305038452148
        Running qft_16.qasm on toyko_20.txt with seed 42
            Swap count: 61, Depth: 101, Runtime: 0.116455078125
        Running qft_20.qasm on toyko_20.txt with seed 42
            Swap count: 113, Depth: 131, Runtime: 0.18312287330627441
        Running rd84_142.qasm on toyko_20.txt with seed 42
            Swap count: 15, Depth: 101, Runtime: 0.13555407524108887
        Running ising_model_10.qasm on toyko_20.txt with seed 42
            Swap count: 0, Depth: 20, Runtime: 0.15352606773376465
        Running ising_model_13.qasm on toyko_20.txt with seed 42
            Swap count: 3, Depth: 41, Runtime: 0.20154285430908203
        Running ising_model_16.qasm on toyko